<a href="https://colab.research.google.com/github/sunjmmeishao/AndroidTest/blob/master/4k_Video_Upscaler_Colab_(Real_ESRGAN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4k Video Upscaler Colab (Real-ESRGAN)

Adapted from: [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN)

Made with ❤️ by: [yuvraj108c](https://github.com/yuvraj108c)

Github repository: https://github.com/yuvraj108c/4k-video-upscaler-colab

# 1. Setup (~1 minute)

In [1]:
import torch
assert torch.cuda.is_available(), "GPU not detected.. Please change runtime to GPU"

from PIL import Image
import cv2, os, subprocess
from tqdm import tqdm

!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN

!pip install -q torch==2.0.1 torchvision==0.15.2 --extra-index-url https://download.pytorch.org/whl/cu118
!pip install -q basicsr facexlib gfpgan ffmpeg ffmpeg-python
!pip install -q -r requirements.txt
!python setup.py develop

!pip install "numpy<2"
mount_drive = False

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 759 (delta 106), reused 98 (delta 98), pack-reused 638 (from 1)
Receiving objects: 100% (759/759), 5.38 MiB | 9.63 MiB/s, done.
Resolving deltas: 100% (415/415), done.
/content/Real-ESRGAN
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 582.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 2.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.0.1+cu118 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

下载

In [11]:
from google.colab import files

# 假设你要下载的文件是 'output.mp4'
files.download('/content/eraser_20250603192258140_upscaled_1440_2560.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
import os

print("当前工作目录：", os.getcwd())  # 显示当前路径
print("目录文件列表：", os.listdir("/content/gdrive/MyDrive"))


当前工作目录： /content/Real-ESRGAN
目录文件列表： ['Google AI Studio', 'Upscaled Videos (REAL-ESRGAN)', '4.mp4']


In [8]:
from google.colab import files

# 假设你要下载的文件是 'output.mp4'
files.download('/content/eraser_20250603192258140_upscaled_1080_1920.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


# 2. Mount drive (optional)

In [14]:
from google.colab import drive
mount_drive=True #@param{type:"boolean"}

if mount_drive:
  drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [10]:
!pip install streamlit
!streamlit run app.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.6 MB/s eta 0:00:00
Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py


# 3. Upscale video

- The upscaled video will be saved to `output_dir`
- If google drive is mounted, it will be also saved at `MyDrive/Upscaled Videos (REAL-ESRGAN)`


In [19]:
video_path="/content/gdrive/MyDrive/4.mp4" #@param{type:"string"}
output_dir="/content/" #@param{type:"string"}
resolution = "2k (2560 x 1440)" # @param ["FHD (1920 x 1080)", "2k (2560 x 1440)", "4k (3840 x 2160)","2 x original", "3 x original", "4 x original"] {type:"string"}
model = "RealESRGAN_x4plus" #@param ["RealESRGAN_x4plus" , "RealESRGAN_x4plus_anime_6B", "realesr-animevideov3"]

assert os.path.exists(video_path), "Video file does not exist"

video_capture = cv2.VideoCapture(video_path)
video_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

final_width = None
final_height = None
aspect_ratio = float(video_width/video_height)

# Get output resolutions
match resolution:
  case "FHD (1920 x 1080)":
    final_width=1920
    final_height=1080
  case "2k (2560 x 1440)":
    final_width=2560
    final_height=1440
  case "4k (3840 x 2160)":
    final_width=3840
    final_height=2160
  case "2 x original":
    final_width=2*video_width
    final_height=2*video_height
  case "3 x original":
    final_width=3*video_width
    final_height=3*video_height
  case "4 x original":
    final_width=4*video_width
    final_height=4*video_height

if aspect_ratio == 1.0 and "original" not in resolution:
  final_height = final_width

if aspect_ratio < 1.0 and "original" not in resolution:
  temp = final_width
  final_width = final_height
  final_height = temp

scale_factor = max(final_width/video_width, final_height/video_height)
isEven = int(video_width * scale_factor) % 2 == 0 and int(video_height * scale_factor) % 2 == 0

# scale_factor needs to be even
while isEven == False:
  scale_factor += 0.01
  isEven = int(video_width * scale_factor) % 2 == 0 and int(video_height * scale_factor) % 2 == 0

print(f"Upscaling from {video_width}x{video_height} to {final_width}x{final_height}, scale_factor={scale_factor}")

!python inference_realesrgan_video.py -n {model} -i '{video_path}' -o '{output_dir}' --outscale {scale_factor}

video_name_with_ext = os.path.basename(video_path)
video_name = video_name_with_ext.replace(".mp4","")
upscaled_video_path = f"{output_dir}{video_name}_out.mp4"
final_video_name = f"{video_name}_upscaled_{final_width}_{final_height}.mp4"
final_video_path = os.path.join(output_dir, final_video_name)

# crop to fit
if "original" not in resolution:
  print("Cropping to fit...")
  command = f"ffmpeg -loglevel error -hwaccel cuda -y -i '{upscaled_video_path}' -c:v h264_nvenc -filter:v  'crop={final_width}:{final_height}:(in_w-{final_width})/2:(in_h-{final_height})/2' -c:v libx264 -pix_fmt yuv420p '{final_video_path}'"
  subprocess.run(command,shell=True)
else:
  # final video path = upscaled video path
  command = f"cp '{upscaled_video_path}' '{final_video_path}'"
  subprocess.run(command,shell=True)

print(f"Upscaled video saved to: {final_video_path}")

# save to drive
if mount_drive:
  drive_folder = "MyDrive/Upscaled Videos (REAL-ESRGAN)"
  save_directory_drive = f"/content/gdrive/{drive_folder}"
  os.makedirs(save_directory_drive, exist_ok=True)

  command = f"cp '{final_video_path}' '{save_directory_drive}/{final_video_name}'"
  subprocess.run(command,shell=True)
  print(f"Saved to drive: /{drive_folder}/{final_video_name}" )

!rm "{upscaled_video_path}"

Upscaling from 720x1280 to 1440x2560, scale_factor=2.0
/usr/local/lib/python3.11/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
You are generating video that is larger than 4K, which will be very slow due to IO speed. We highly recommend to decrease the outscale(aka, -s).
inference: 100% 153/153 [15:22<00:00,  6.03s/frame]
Cropping to fit...
Upscaled video saved to: /content/4_upscaled_1440_2560.mp4
Saved to drive: /MyDrive/Upscaled Videos (REAL-ESRGAN)/4_upscaled_1440_2560.mp4


# 4. Disconnect runtime

In [ ]:
from google.colab import runtime

disconnect_when_finish = False  #@param{type:"boolean"}

if disconnect_when_finish:
  runtime.unassign()